# Data Cleaning
To clean this data set we'll start out by loading the dataset, checking for duplicates, and dropping columns that are not relevant to our analysis.

First, we'll load in our packages, set up our directories, and load in the dataset and take a look at it.

In [1]:
import pandas as pd
from pathlib import Path

#Set up directories
data_dir = Path('../data')
input_dir = data_dir / 'input'
output_dir = data_dir / 'output'

df = pd.read_csv(input_dir / 'phase2_sample.csv').set_index('Unnamed: 0')
df.index.names = ['Index']
df.head()

,indexed,reference-count,publisher,issue,license,content-domain,short-container-title,published-print,DOI,type,created,page,source,is-referenced-by-count,title,prefix,volume,author,member,reference,container-title,language,link,deposited,score,resource,issued,references-count,journal-issue,URL,ISSN,issn-type,subject,published,alternative-id,published-online,archive,update-policy,assertion,funder,article-number,accepted,abstract,original-title,subtitle,published-other,editor
Index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,"{'date-parts': [[2022, 10, 7]], 'date-time': '...",14,Wiley,1,"[{'start': {'date-parts': [[2015, 9, 1]], 'dat...","{'domain': [], 'crossmark-restriction': False}",['Syst. Dyn. Rev.'],{'date-parts': [[2000]]},10.1002/(sici)1099-1727(200021)16:1<27::aid-sd...,journal-article,"{'date-parts': [[2002, 9, 10]], 'date-time': '...",27-41,Crossref,57,['The validation of commercial system dynamics...,10.1002,16,"[{'given': 'Geoff', 'family': 'Coyle', 'sequen...",311.0,[{'key': '10.1002/(SICI)1099-1727(200021)16:1<...,['System Dynamics Review'],en,[{'URL': 'https://api.wiley.com/onlinelibrary/...,"{'date-parts': [[2021, 7, 1]], 'date-time': '2...",0.0,{'primary': {'URL': 'https://onlinelibrary.wil...,{'date-parts': [[2000]]},14,"{'issue': '1', 'published-print': {'date-parts...",http://dx.doi.org/10.1002/(sici)1099-1727(2000...,"['0883-7066', '1099-1727']","[{'value': '0883-7066', 'type': 'print'}, {'va...","['Management of Technology and Innovation', 'S...",{'date-parts': [[2000]]},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'date-parts': [[2022, 3, 29]], 'date-time': '...",12,Springer Science and Business Media LLC,1,"[{'start': {'date-parts': [[1979, 3, 1]], 'dat...","{'domain': [], 'crossmark-restriction': False}",['MTB'],"{'date-parts': [[1979, 3]]}",10.1007/bf02653972,journal-article,"{'date-parts': [[2007, 7, 17]], 'date-time': '...",57-62,Crossref,20,['Effect of system geometry on the leaching be...,10.1007,10,"[{'given': 'C.', 'family': 'Vu', 'sequence': '...",297.0,"[{'key': 'BF02653972_CR1', 'volume-title': 'Ph...",['Metallurgical Transactions B'],en,[{'URL': 'http://link.springer.com/content/pdf...,"{'date-parts': [[2019, 5, 20]], 'date-time': '...",0.0,{'primary': {'URL': 'http://link.springer.com/...,"{'date-parts': [[1979, 3]]}",12,"{'issue': '1', 'published-print': {'date-parts...",http://dx.doi.org/10.1007/bf02653972,"['0360-2141', '1543-1916']","[{'value': '0360-2141', 'type': 'print'}, {'va...","['Materials Chemistry', 'Metals and Alloys', '...","{'date-parts': [[1979, 3]]}",['BF02653972'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'date-parts': [[2022, 3, 30]], 'date-time': '...",0,Wiley,3,"[{'start': {'date-parts': [[2017, 11, 1]], 'da...","{'domain': [], 'crossmark-restriction': False}",['RECIEL'],"{'date-parts': [[2017, 11]]}",10.1111/reel.12221,journal-article,"{'date-parts': [[2017, 12, 1]], 'date-time': '...",243-254,Crossref,2,['The international law on transboundary haze ...,10.1111,26,"[{'given': 'Shawkat', 'family': 'Alam', 'seque...",311.0,NaN,"['Review of European, Comparative &amp; Intern...",en,[{'URL': 'https://api.wiley.com/onlinelibrary/...,"{'date-parts': [[2017, 12, 1]], 'date-time': '...",0.0,{'primary': {'URL': 'http://doi.wiley.com/10.1...,"{'date-parts': [[2017, 11]]}",0,"{'issue': '3', 'published-print': {'date-parts...",http://dx.doi.org/10.1111/reel.12221,['2050-0386'],"[{'value': '2050-0386', 'type': 'print'}]","['Law', 'Management, Monitoring, Policy and La...","{'date-parts': [[2017, 11]]}",NaN,"{'date-parts': [[2017, 11, 28]]}",['Portico'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'date-parts': [[2022, 4, 3]], 'date-time': '2...",0,Crop Science Society of Japan,1-2,NaN,"{'domain': [], 'crossmark-restriction': False}","['Japanese journal of crop science', 'Jpn. J. ...",{'date-parts': [[1951]]},10.1626/jcs.20.219,journal-article,"{'date-parts': [[2011, 9, 20]], 'date-time': '...",219-222,Crossref,0,['Studies on the influence of pruning on the 

## Duplicate Records
Looking at the shape of the dataset against the number of unique DOIs will let us know just how many duplicate records we have.


In [2]:
df.shape

(100107, 47)

In [3]:
len(set(df['DOI']))

100044

In [4]:
#Dropping duplicate records
df.drop_duplicates(subset=['DOI'], keep='first', inplace=True)
df.shape

(100044, 47)

## Missing values and Irrelevant Columns
We're going to see just how many records are missing data and, considering the scope and objectives of our analysis, we'll drop some of the columns that we won't be using.

In [5]:
#Looking at null values by column
missing_values_count = df.isnull().sum()
missing_values_count

indexed                       0
reference-count               0
publisher                    22
issue                     11940
license                   46166
content-domain                0
short-container-title     10168
published-print           14778
DOI                           0
type                          0
created                       0
page                       9106
source                        0
is-referenced-by-count        0
title                       303
prefix                        0
volume                     4924
author                    10268
member                       22
reference                 47983
container-title               0
language                  20170
link                      16459
deposited                     0
score                         0
resource                      1
issued                        0
references-count              0
journal-issue             11922
URL                           0
ISSN                        338
issn-typ

Some of these columns with a high number of null values are of particular interest to us, such as the *abstract* column, so we'll be sure to keep that one. Most of the columns towards to end we won't be using so we'll drop them just to make our cleaned dataset a little less cumbersome. Additionally, there are other columns with few null values that we will drop for much the same reason, such as *score, published, ISSN*. We will also drop columns like *source* and *type* because all the records have the same *source: Crossref* and the same *type: journal-article*.

In [6]:
df.source.unique()

array(['Crossref'], dtype=object)

In [7]:
df.type.unique()

array(['journal-article'], dtype=object)

In [8]:
#Dropping some of the unnnecessary columns
dropped_cols = ['funder', 'accepted', 'article-number', 'archive', 'published-other', 'issue', 'type', 'ISSN', 'issn-type',
               'reference-count', 'score', 'published-online', 'published', 'published-print', 'update-policy', 'source', 'page', 'license']
df.drop(columns=dropped_cols, inplace=True)


## Editors
There are very few records that have a value in the *editor* column. Some of our prior work indicates that this can be a sign of a work that has been mislabeled as a 'journal article'. So we'll explore some of the records with a value in the editor column in order to verify that.

We'll set up a dataframe of just those records that have data in the *editor* column.

In [9]:
editor_df = df.loc[df.editor.notna()]
editor_df.head()

,indexed,publisher,content-domain,short-container-title,DOI,created,is-referenced-by-count,title,prefix,volume,author,member,reference,container-title,language,link,deposited,resource,issued,references-count,journal-issue,URL,subject,alternative-id,assertion,abstract,original-title,subtitle,editor
Index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
44,"{'date-parts': [[2022, 10, 26]], 'date-time': ...",Public Library of Science (PLoS),"{'domain': ['www.plosone.org'], 'crossmark-res...",['PLoS ONE'],10.1371/journal.pone.0089413,"{'date-parts': [[2014, 3, 5]], 'date-time': '2...",77,['Heritable Multiplex Genetic Engineering in R...,10.1371,9,"[{'given': 'Yuanwu', 'family': 'Ma', 'sequence...",340.0,"[{'key': 'ref1', 'doi-asserted-by': 'crossref'...",['PLoS ONE'],en,[{'URL': 'http://dx.plos.org/10.1371/journal.p...,"{'date-parts': [[2020, 5, 8]], 'date-time': '2...",{'primary': {'URL': 'https://dx.plos.org/10.13...,"{'date-parts': [[2014, 3, 5]]}",24,"{'issue': '3', 'published-online': {'date-part...",http://dx.doi.org/10.1371/journal.pone.0089413,['Multidisciplinary'],NaN,NaN,NaN,NaN,NaN,"[{'given': 'Xiaolei', 'family': 'Xu', 'sequenc..."
117,"{'date-parts': [[2022, 4, 2]], 'date-time': '2...",EDP Sciences,"{'domain': [], 'crossmark-restriction': False}",['EPJ Web Conf.'],10.1051/epjconf/202124801022,"{'date-parts': [[2021, 4, 26]], 'date-time': '...",0,['Distributions of Two Atoms Collisions over t...,10.1051,248,"[{'given': 'Sergey', 'family': 'Zheltov', 'seq...",250.0,"[{'key': 'R1', 'doi-asserted-by': 'crossref', ...",['EPJ Web of Conferences'],NaN,[{'URL': 'https://www.epj-conferences.org/10.1...,"{'date-parts': [[2021, 4, 26]], 'date-time': '...",{'primary': {'URL': 'https://www.epj-conferenc...,{'date-parts': [[2021]]},17,NaN,http://dx.doi.org/10.1051/epjconf/202124801022,"['General Earth and Planetary Sciences', 'Gene...",['epjconf_mnps2021_01022'],NaN,<jats:p>The processes of heat and mass transfe...,NaN,NaN,"[{'given': 'A.', 'family': 'Nadykto', 'sequenc..."
126,"{'date-parts': [[2022, 8, 19]], 'date-time': '...",Public Library of Science (PLoS),"{'domain': ['www.plosone.org'], 'crossmark-res...",['PLoS ONE'],10.1371/journal.pone.0030827,"{'date-parts': [[2012, 2, 2]], 'date-time': '2...",44,['Transcriptional Analysis of Shewanella oneid...,10.1371,7,"[{'given': 'Miriam A.', 'family': 'Rosenbaum',...",340.0,"[{'key': 'ref1', 'doi-asserted-by': 'crossref'...",['PLoS ONE'],en,[{'URL': 'http://dx.plos.org/10.1371/journal.p...,"{'date-parts': [[2018, 10, 23]], 'date-time': ...",{'primary': {'URL': 'https://dx.plos.org/10.13...,"{'date-parts': [[2012, 2, 1]]}",58,"{'issue': '2', 'published-online': {'date-part...",http://dx.doi.org/10.1371/journal.pone.0030827,['Multidisciplinary'],NaN,NaN,NaN,NaN,NaN,"[{'given': 'Pierre', 'family': 'Cornelis', 'se..."
138,"{'date-parts': [[2022, 10, 22]], 'date-time': ...",Public Library of Science (PLoS),"{'domain': ['www.plosone.org'], 'crossmark-res...",['PLoS ONE'],10.1371/journal.pone.0053057,"{'date-parts': [[2013, 1, 16]], 'date-time': '...",28,['The Influence of Vibratory Courtship on Fema...,10.1371,8,"[{'given': 'Anne E.', 'family': 'Wignall', 'se...",340.0,"[{'key': 'ref1', 'doi-asserted-by': 'crossref'...",['PLoS ONE'],en,[{'URL': 'http://dx.plos.org/10.1371/journal.p...,"{'date-parts': [[2022, 2, 6]], 'date-time': '2...",{'primary': {'URL': 'https://dx.plos.org/10.13...,"{'date-parts': [[2013, 1, 16]]}",43,"{'issue': '1', 'published-online': {'date-part...",http://dx.doi.org/10.1371/journal.pone.0053057,['Multidisciplinary'],NaN,NaN,NaN,NaN,NaN,"[{'given': 'Jan M.', 'family': 'Hemmi', 'seque..."
139,"{'date-parts': [[2022, 8, 24]], 'date-time': '...",Emerald,"{'domain': [], 'crossmark-restriction': False}",['critical perspect on int bus'],10.1108/17422040810869972,"{'date-parts': [[2008, 6, 14]], 'date-time': '...",31,['We are all stakeholders now'],10.1108,4,"[{'given': 'Richard', 'family': 'Parsons', 'se...",140.0,"[{'key': 'b1', 'doi-asserted-by': 'publisher',...",['Critical perspectives on

Next, we'll search the titles of these records for a few keywords.

In [10]:
editorial = editor_df.loc[editor_df.title.str.contains(r'editorial|errata|contents|conference|proceedings', regex=True, case=False)]
editorial

,indexed,publisher,content-domain,short-container-title,DOI,created,is-referenced-by-count,title,prefix,volume,author,member,reference,container-title,language,link,deposited,resource,issued,references-count,journal-issue,URL,subject,alternative-id,assertion,abstract,original-title,subtitle,editor
Index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
25040,"{'date-parts': [[2022, 4, 5]], 'date-time': '2...",SAGE Publications,"{'domain': [], 'crossmark-restriction': False}","['Illness, Crisis &amp; Loss']",10.2190/il.17.2.a,"{'date-parts': [[2009, 3, 20]], 'date-time': '...",0,['Editorial'],10.1177,17,NaN,179.0,NaN,"['Illness, Crisis &amp; Loss']",en,[{'URL': 'http://journals.sagepub.com/doi/pdf/...,"{'date-parts': [[2021, 5, 29]], 'date-time': '...",{'primary': {'URL': 'http://journals.sagepub.c...,"{'date-parts': [[2009, 3, 19]]}",0,"{'issue': '2', 'published-print': {'date-parts...",http://dx.doi.org/10.2190/il.17.2.a,"['Sociology and Political Science', 'Health (s...",['10.2190/IL.17.2.a'],NaN,NaN,NaN,NaN,"[{'given': 'Neil', 'family': 'Thompson', 'sequ..."
52118,"{'date-parts': [[2022, 4, 2]], 'date-time': '2...",EDP Sciences,"{'domain': [], 'crossmark-restriction': False}",['EPJ Web Conf.'],10.1051/epjconf/201613100001,"{'date-parts': [[2016, 12, 1]], 'date-time': '...",4,['Editorial'],10.1051,131,"[{'given': 'Dirk', 'family': 'Rudolph', 'seque...",250.0,"[{'key': 'R1', 'unstructured': 'https://www.no...",['EPJ Web of Conferences'],NaN,[{'URL': 'http://www.epj-conferences.org/10.10...,"{'date-parts': [[2017, 1, 6]], 'date-time': '2...",{'primary': {'URL': 'http://www.epj-conference...,{'date-parts': [[2016]]},4,NaN,http://dx.doi.org/10.1051/epjconf/201613100001,"['General Earth and Planetary Sciences', 'Gene...",['epjconf-NS160-00001'],NaN,NaN,NaN,NaN,"[{'given': 'D.', 'family': 'Rudolph', 'sequenc..."
76238,"{'date-parts': [[2022, 4, 1]], 'date-time': '2...",Association for Computing Machinery (ACM),"{'domain': [], 'crossmark-restriction': False}",['ACM Trans. Graph.'],10.1145/944020.944021,"{'date-parts': [[2003, 10, 10]], 'date-time': ...",0,['Editorial'],10.1145,22,NaN,320.0,NaN,['ACM Transactions on Graphics'],en,[{'URL': 'http://dl.acm.org/ft_gateway.cfm?id=...,"{'date-parts': [[2020, 10, 4]], 'date-time': '...",{'primary': {'URL': 'https://dl.acm.org/doi/10...,"{'date-parts': [[2003, 10]]}",0,"{'issue': '4', 'published-print': {'date-parts...",http://dx.doi.org/10.1145/944020.944021,['Computer Graphics and Computer-Aided Design'],['10.1145/944020.944021'],NaN,NaN,NaN,NaN,"[{'given': 'John C.', 'family': 'Hart', 'seque..."
78072,"{'date-parts': [[2022, 4, 2]], 'date-time': '2...",Oxford University Press (OUP),"{'domain': [], 'crossmark-restriction': False}",NaN,10.1093/ajhp/2.6.176,"{'date-parts': [[2019, 1, 10]], 'date-time': '...",0,['Editorial'],10.1093,2,NaN,286.0,NaN,['American Journal of Health-System Pharmacy'],en,[{'URL': 'http://academic.oup.com/ajhp/article...,"{'date-parts': [[2019, 3, 31]], 'date-time': '...",{'primary': {'URL': 'https://academic.oup.com/...,"{'date-parts': [[1945, 12, 1]]}",0,"{'issue': '6', 'published-print': {'date-parts...",http://dx.doi.org/10.1093/ajhp/2.6.176,"['Health Policy', 'Pharmacology']",NaN,NaN,NaN,NaN,NaN,"[{'given': 'Anna D.', 'family': 'Thiel', 'sequ..."


We've found a handful of Editorials. We'll go ahead and drop them from our dataset.

In [11]:
df.drop(editorial.index, inplace=True)

In [12]:
df.shape

(100040, 29)

## Conferences
Looking back at the **editor_df** we see that there are a couple 'Conferences' and 'Proceedings' in the *container-title* column. Let's take a look at just how many records in our dataset are from these journals/containers.

We'll use a keyword search in the *container-title* column to find these records.

In [13]:
conferences = df.loc[df['container-title'].str.contains(r'conference', regex=True, case=False)]
conferences

,indexed,publisher,content-domain,short-container-title,DOI,created,is-referenced-by-count,title,prefix,volume,author,member,reference,container-title,language,link,deposited,resource,issued,references-count,journal-issue,URL,subject,alternative-id,assertion,abstract,original-title,subtitle,editor
Index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
57,"{'date-parts': [[2022, 3, 31]], 'date-time': '...",IOP Publishing,"{'domain': ['iopscience.iop.org'], 'crossmark-...",['IOP Conf. Ser.: Earth Environ. Sci.'],10.1088/1755-1315/660/1/012131,"{'date-parts': [[2021, 2, 22]], 'date-time': '...",0,['Application of wavelet multi-scale analysis ...,10.1088,660,"[{'given': 'Hailong', 'family': 'Sun', 'sequen...",266.0,"[{'key': 'EES_660_1_012131bib1', 'author': 'Ha...",['IOP Conference Series: Earth and Environment...,NaN,[{'URL': 'https://iopscience.iop.org/article/1...,"{'date-parts': [[2022, 1, 29]], 'date-time': '...",{'primary': {'URL': 'https://iopscience.iop.or...,"{'date-parts': [[2021, 2, 1]]}",6,"{'issue': '1', 'published-print': {'date-parts...",http://dx.doi.org/10.1088/1755-1315/660/1/012131,NaN,NaN,[{'value': 'Application of wavelet multi-scale...,<jats:title>Abstract</jats:title>\n ...,NaN,NaN,NaN
117,"{'date-parts': [[2022, 4, 2]], 'date-time': '2...",EDP Sciences,"{'domain': [], 'crossmark-restriction': False}",['EPJ Web Conf.'],10.1051/epjconf/202124801022,"{'date-parts': [[2021, 4, 26]], 'date-time': '...",0,['Distributions of Two Atoms Collisions over t...,10.1051,248,"[{'given': 'Sergey', 'family': 'Zheltov', 'seq...",250.0,"[{'key': 'R1', 'doi-asserted-by': 'crossref', ...",['EPJ Web of Conferences'],NaN,[{'URL': 'https://www.epj-conferences.org/10.1...,"{'date-parts': [[2021, 4, 26]], 'date-time': '...",{'primary': {'URL': 'https://www.epj-conferenc...,{'date-parts': [[2021]]},17,NaN,http://dx.doi.org/10.1051/epjconf/202124801022,"['General Earth and Planetary Sciences', 'Gene...",['epjconf_mnps2021_01022'],NaN,<jats:p>The processes of heat and mass transfe...,NaN,NaN,"[{'given': 'A.', 'family': 'Nadykto', 'sequenc..."
140,"{'date-parts': [[2022, 11, 8]], 'date-time': '...",Association for the Advancement of Artificial ...,"{'domain': [], 'crossmark-restriction': False}",['AAAI'],10.1609/aaai.v32i1.11721,"{'date-parts': [[2022, 6, 24]], 'date-time': '...",6,['SC2Net: Sparse LSTMs for Sparse Coding'],10.1609,32,"[{'given': 'Joey Tianyi', 'family': 'Zhou', 's...",9382.0,NaN,['Proceedings of the AAAI Conference on Artifi...,NaN,[{'URL': 'https://ojs.aaai.org/index.php/AAAI/...,"{'date-parts': [[2022, 11, 7]], 'date-time': '...",{'primary': {'URL': 'https://ojs.aaai.org/inde...,"{'date-parts': [[2018, 4, 29]]}",0,"{'issue': '1', 'published-online': {'date-part...",http://dx.doi.org/10.1609/aaai.v32i1.11721,['General Medicine'],NaN,NaN,<jats:p>\n \n The iterative hard-t...,NaN,NaN,NaN
229,"{'date-parts': [[2022, 3, 29]], 'date-time': '...",EDP Sciences,"{'domain': [], 'crossmark-restriction': False}",['E3S Web Conf.'],10.1051/e3sconf/201910502010,"{'date-parts': [[2019, 6, 21]], 'date-time': '...",1,['Computer Simulation of the Physical Processe...,10.1051,105,"[{'given': 'Maxim', 'family': 'Gucal', 'sequen...",250.0,"[{'key': 'R1', 'doi-asserted-by': 'crossref', ...",['E3S Web of Conferences'],NaN,[{'URL': 'https://www.e3s-conferences.org/10.1...,"{'date-parts': [[2020, 3, 13]], 'date-time': '...",{'primary': {'URL': 'https://www.e3s-conferenc...,{'date-parts': [[2019]]},13,NaN,http://dx.doi.org/10.1051/e3sconf/201910502010,"['Pulmonary and Respiratory Medicine', 'Pediat...",['e3sconf_iims18_02010'],NaN,<jats:p>An algorithm for determining the elect...,NaN,NaN,"[{'given': 'M.', 'family': 'Tyulenev', 'sequen..."
562,"{'date-parts': [[2022, 3, 29]], 'date-time': '...",Rezekne Academy of Technologies,"{'domain': [], 'crossmark-restriction': False}",['AMCD'],10.17770/amcd2013.1271,"{'date-parts': [[2016, 4, 26]], 'date-time': '...",0,['Representation of Latvian Mithology in works...,10.1777,NaN,"[{'given': 'Austra', 'family': 'Celmiņa-Ķei

In [14]:
df.drop(conferences.index, inplace=True)

## Cleaning Dates
Here we are going to re-format some of the datetime columns into a more easily parsed format. *Indexed, created,* and *deposited*.

We'll use a regular expression to extract the dates from each of the records in each of those three columns, then we'll convert them to datetime dtypes.

In [15]:
date_columns = ['indexed', 'created', 'deposited']

for col in date_columns:
    df[col] = df[col].str.extract(r"\'(\d{4}\S\d{2}\S\d{2})")

In [16]:
df['indexed'] = pd.to_datetime(df['indexed'], format="%Y-%m-%d")
df['created'] = pd.to_datetime(df['created'], format="%Y-%m-%d")
df['deposited'] = pd.to_datetime(df['deposited'], format="%Y-%m-%d")

df.head()

,indexed,publisher,content-domain,short-container-title,DOI,created,is-referenced-by-count,title,prefix,volume,author,member,reference,container-title,language,link,deposited,resource,issued,references-count,journal-issue,URL,subject,alternative-id,assertion,abstract,original-title,subtitle,editor
Index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2022-10-07,Wiley,"{'domain': [], 'crossmark-restriction': False}",['Syst. Dyn. Rev.'],10.1002/(sici)1099-1727(200021)16:1<27::aid-sd...,2002-09-10,57,['The validation of commercial system dynamics...,10.1002,16,"[{'given': 'Geoff', 'family': 'Coyle', 'sequen...",311.0,[{'key': '10.1002/(SICI)1099-1727(200021)16:1<...,['System Dynamics Review'],en,[{'URL': 'https://api.wiley.com/onlinelibrary/...,2021-07-01,{'primary': {'URL': 'https://onlinelibrary.wil...,{'date-parts': [[2000]]},14,"{'issue': '1', 'published-print': {'date-parts...",http://dx.doi.org/10.1002/(sici)1099-1727(2000...,"['Management of Technology and Innovation', 'S...",NaN,NaN,NaN,NaN,NaN,NaN
1,2022-03-29,Springer Science and Business Media LLC,"{'domain': [], 'crossmark-restriction': False}",['MTB'],10.1007/bf02653972,2007-07-17,20,['Effect of system geometry on the leaching be...,10.1007,10,"[{'given': 'C.', 'family': 'Vu', 'sequence': '...",297.0,"[{'key': 'BF02653972_CR1', 'volume-title': 'Ph...",['Metallurgical Transactions B'],en,[{'URL': 'http://link.springer.com/content/pdf...,2019-05-20,{'primary': {'URL': 'http://link.springer.com/...,"{'date-parts': [[1979, 3]]}",12,"{'issue': '1', 'published-print': {'date-parts...",http://dx.doi.org/10.1007/bf02653972,"['Materials Chemistry', 'Metals and Alloys', '...",['BF02653972'],NaN,NaN,NaN,NaN,NaN
2,2022-03-30,Wiley,"{'domain': [], 'crossmark-restriction': False}",['RECIEL'],10.1111/reel.12221,2017-12-01,2,['The international law on transboundary haze ...,10.1111,26,"[{'given': 'Shawkat', 'family': 'Alam', 'seque...",311.0,NaN,"['Review of European, Comparative &amp; Intern...",en,[{'URL': 'https://api.wiley.com/onlinelibrary/...,2017-12-01,{'primary': {'URL': 'http://doi.wiley.com/10.1...,"{'date-parts': [[2017, 11]]}",0,"{'issue': '3', 'published-print': {'date-parts...",http://dx.doi.org/10.1111/reel.12221,"['Law', 'Management, Monitoring, Policy and La...",NaN,NaN,NaN,NaN,NaN,NaN
3,2022-04-03,Crop Science Society of Japan,"{'domain': [], 'crossmark-restriction': False}","['Japanese journal of crop science', 'Jpn. J. ...",10.1626/jcs.20.219,2011-09-20,0,['Studies on the influence of pruning on the v...,10.1626,20,"[{'given': 'C.', 'family': 'TSUDA', 'sequence'...",632.0,NaN,['Japanese Journal of Crop Science'],en,[{'URL': 'http://www.jstage.jst.go.jp/article/...,2021-04-30,{'primary': {'URL': 'http://www.jstage.jst.go....,{'date-parts': [[1951]]},0,"{'issue': '1-2', 'published-print': {'date-par...",http://dx.doi.org/10.1626/jcs.20.219,"['Genetics', 'Agronomy and Crop Science', 'Foo...",NaN,NaN,NaN,NaN,NaN,NaN
4,2022-03-31,Elsevier BV,"{'domain': ['clinicalkey.fr', 'elsevier.com', ...",['Revue de Pneumologie Clinique'],10.1016/j.pneumo.2018.09.002,2018-10-10,0,['Le tabagisme et l’aide à l’arrêt du tabac de...,10.1016,74,"[{'given': 'J.', 'family': 'Perriot', 'sequenc...",78.0,[{'key': '10.1016/j.pneumo.2018.09.002_bib0305...,['Revue de Pneumologie Clinique'],fr,[{'URL': 'https://api.elsevier.com/content/art...,2019-10-26,{'primary': {'URL': 'https://linkinghub.elsevi...,"{'date-parts': [[2018, 12]]}",60,"{'issue': '6', 'published-print': {'date-parts...",http://dx.doi.org/10.1016/j.pneumo.2018.09.002,['Pulmonary and Respiratory Medicine'],['S0761841718301792'],"[{'value': 'Elsevier', 'name': 'publisher', 'l...",NaN,NaN,NaN,NaN


# String slicing
Now that the dates are converted, one of the last problems to address are some of the excess character in the *title, short-container-title,* and *container-title* fields.

In [17]:
cols = ['title', 'short-container-title', 'container-title']
for col in cols:
    df[col] = df[col].str.slice(start=2, stop=-2)

In [18]:
df['title'][0]

'The validation of commercial system dynamics models'

## Cleaning XML tags
We'll be looking at the abstract column, so it will benefit us to clean out the tags and only have te relevant text for each record. We'll write a quick function to do that.

In [22]:
#import beatuiful soup
from bs4 import BeautifulSoup as bs
def clean_abstracts(abstract):
    try:
        soup = bs(abstract, features='lxml')
        stripped_strings = soup.get_text()
        return stripped_strings
    except:
        return None
stripped_abstracts = df.abstract.map(lambda x: clean_abstracts(x))
df['abstract'] = stripped_abstracts

In [35]:
df['abstract'][11]

'A novel and efficient approach to the synthesis of 2-vinylbenzylamines is reported. This involves obtaining 2-vinylbenzylamine ligands from tetrahydroisoquinoline by alkylation and reduction followed by the Hofmann cleavage. The resultant 2-vinylbenzylamines allowed us to obtain new Hoveyda–Grubbs catalysts, which were thoroughly characterised by NMR, ESIMS, and X-ray crystallography. The utility of this chemistry is further demonstrated by the tests of the novel catalysts (up to 10−2 mol %) in different metathesis reactions such as cross metathesis (CM), ring-closing metathesis (RCM) and ring-opening cross metathesis (ROCM).'

Looks great! Now we'll save our cleaned dataset.

In [36]:
df.to_csv(input_dir / 'cleaned_data.csv')